In [ ]:
## set covid19_datasciente as path for python find bulletin package
import sys
from pathlib import Path
from os import getcwd, remove, chdir
from os.path import join, basename

sys.path.append(str(Path(getcwd()).parent))
current_dir = getcwd()
print(current_dir)

In [ ]:
from bulletin.systems import Notifica

In [ ]:
nt = Notifica()
nt.load('notifica', False)
nt.df.shape

In [ ]:
curitiba = nt.df.loc[nt.df['ibge_residencia']==410690].copy()
curitiba.shape

In [ ]:
curitiba.groupby(['classificacao_final'])[['id']].count().rename(index=nt.tables['classificacao_final'].set_index('id')['classificacao_final'])

In [ ]:
curitiba['status'] = 0
curitiba.loc[curitiba['status_notificacao'].isin([1,2]),'status'] = 1

curitiba.groupby(['status','classificacao_final'])[['id']].count().rename(index=nt.tables['classificacao_final'].set_index('id')['classificacao_final'],level='classificacao_final').rename(index={0:'Inativa/Duplicada',1:'Aberta/Encerrada'},level='status')

In [ ]:
curitiba.groupby(['status','excluir_ficha','classificacao_final'])[['id']].count().rename(index=nt.tables['classificacao_final'].set_index('id')['classificacao_final'],level='classificacao_final').rename(index=nt.tables['sim_nao'].set_index('id')['sim_nao'],level='excluir_ficha').rename(index={0:'Inativa/Duplicada',1:'Aberta/Encerrada'},level='status')

In [ ]:
curitiba.loc[curitiba['reinfeccao'].isna(),'reinfeccao'] = False
curitiba.loc[(curitiba['status']==1) & (curitiba['excluir_ficha']==2) & (curitiba['classificacao_final']==2)].groupby(['reinfeccao'])[['id']].count().rename(columns={'id':'casos confirmados'})